In [3]:
# Import the API
import sys
sys.path.append('path/to/okx/operator/src')

from src.main_engine import build_expr, EvalContext,build_optimized_expr,build_multiple_expr
import polars as pl
data = pl.scan_parquet(r'E:\Random_Project\okx\operator\test\data\financial_test_data.parquet')
# Create context
ctx = EvalContext(
    variables={
        "close": pl.col("close"),
        "volume": pl.col("volume"),
        "returns": pl.col("returns")
    },
    ticker_col="ticker",
    time_col="time"
)


In [4]:
expr = build_expr("TSMean(close, 20)", ctx)
result = data.with_columns(factor=expr).collect()

In [5]:
# Complex nested expression - automatically handled
result = build_optimized_expr('CSScale(Add(Clip(turnover, 0.0, 10.0), TSLag(TSMean(Log(turnover), 20), 1)))', ctx, data, "complex_alpha")
final = result.select(['ticker', 'time', 'complex_alpha']).collect()

In [6]:
final

ticker,time,complex_alpha
str,i64,f64
"""AAPL""",0,null
"""AMZN""",0,0.88707
"""BABA""",0,0.198547
"""BIDU""",0,-1.171392
"""CRM""",0,0.322099
…,…,…
"""NFLX""",6047,0.121564
"""NIO""",6047,0.838984
"""NVDA""",6047,-0.380725


In [7]:
expressions = {
    'ma20': 'TSMean(close, 20)',
    'volatility_': 'TSStd(returns, 20)',
    'complex_nested': 'TSMean(TSStd(close, 10), 5)',     # Complex nesting - works!
    'mixed_operators': 'CSRank(TSMean(volume, 10))',     # Mixed operators - works!
    'time_on_cross': 'TSMean(CSScale(returns), 5)'       # Time series on cross section - works!
}

builder = build_multiple_expr(expressions, ctx)
result = builder.apply(data.lazy()).collect()

print(f"Successfully built {len([col for col in result.columns if col in expressions.keys()])} factors")

Successfully built 5 factors


In [8]:
result

ticker,time,high,amount,pct_change,spread,low,close,open,vwap,turnover,returns,relative_position,volume,ma20,volatility_,complex_nested,mixed_operators,time_on_cross
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""AAPL""",0,239.304905,3.5666e12,0.0,27.537063,211.767842,211.825978,238.885503,225.477106,3.5666e9,0.000128,0.002111,1.6910e7,211.825978,null,null,55525.0,0.233248
"""AMZN""",0,315.95973,8.0362e12,0.0,24.591651,291.36808,315.686143,291.562023,303.693425,8.0362e9,0.000899,0.988875,2.5414e7,315.686143,null,null,67522.0,0.6912
"""BABA""",0,60.799873,1.1318e11,0.0,5.310259,55.489614,60.746874,55.575639,58.179233,1.1318e8,0.000416,0.99002,1.8486e6,60.746874,null,null,14691.0,0.652564
"""BIDU""",0,425.656933,1.1650e12,0.0,5.219194,420.437739,421.35043,425.634732,422.89839,1.1650e9,-0.000346,0.174872,2.8417e6,421.35043,null,null,19521.0,0.406759
"""CRM""",0,368.735851,7.6041e12,0.0,7.158102,361.577749,368.235522,362.400385,365.505631,7.6041e9,0.000788,0.930103,2.1083e7,368.235522,null,null,61920.0,0.528884
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NFLX""",6047,1.0638e8,3.3952e16,-7694.276424,6.3061e6,1.0008e8,1.0622e8,1.0023e8,1.0308e8,3.3952e13,-0.030282,0.974259,324364.626345,9.8964e7,0.053008,421643.594705,3653.0,-0.59229
"""NIO""",6047,495082.352578,4.0964e15,-172.452656,73805.950579,421276.401999,421727.606227,494941.571187,458276.682074,4.0964e12,-0.054929,0.006113,9.4597e6,475059.23386,0.033036,424071.755086,24486.0,-0.847755
"""NVDA""",6047,361.308255,4.3670e11,0.095443,61.671566,299.636689,358.376688,305.629,331.029632,4.3670e8,0.032948,0.952465,1.2309e6,365.493551,0.026804,424073.520879,9283.0,-0.043412
